### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth==2025.3.19 vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth==2025.3.19 vllm
!pip install --no-deps transformers==4.50.3

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo==2025.3.17
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

In [3]:
from unsloth import PatchDPOTrainer

PatchDPOTrainer()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-15 07:38:51 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-15 07:38:51 [__init__.py:239] Automatically detected platform cuda.


In [4]:
from unsloth import FastLanguageModel

max_seq_length = 512
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

<a name="Data"></a>
### Data Prep

We first download the data files.

In [5]:
!git clone https://gitlab.com/lchengtw/ML2025Spring-HW7.git

Cloning into 'ML2025Spring-HW7'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9 (from 1)
Receiving objects: 100% (9/9), 8.63 KiB | 2.16 MiB/s, done.
Resolving deltas: 100% (1/1), done.


Then, we load the json file here.

In [6]:
import json

with open("/content/ML2025Spring-HW7/train.json", 'r') as jsonfile:
    full_data = json.load(jsonfile)

with open("/content/ML2025Spring-HW7/test.json", 'r') as jsonfile:
    test_data = json.load(jsonfile)

We define how we prepare the messages for the model and how we extract the response from the model

In [7]:
import re

def data_formulate(data):
    messages = [
        {"role": "system", "content": "Your entire response must be 100 characters or less."},
        {"role": "user", "content": data['prompt']},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

def extract_assistant_response(text):
    try:
        # Split by assistant header marker
        parts = text.split("<|start_header_id|>assistant<|end_header_id|>")
        if len(parts) < 2:
            return None

        # Split by end of text marker
        assistant_part = parts[1]
        response_parts = assistant_part.split("<|eot_id|>")

        # Clean up any whitespace
        return response_parts[0].strip()
    except Exception as e:
        print(f"Error extracting assistant response: {e}")
        return None

Let's observe how the model responses before aligning it.

In [8]:
original_model_response = []
for data in test_data:
    id = data['id']
    prompt = data['prompt']
    print(f'\nQuestion {id}: {prompt}')
    inputs = data_formulate(data)
    outputs = model.generate(
        **tokenizer(inputs, return_tensors = "pt").to("cuda"),
        max_new_tokens = 128,
        do_sample=False
    )
    output = tokenizer.batch_decode(outputs)[0]
    output = extract_assistant_response(output)
    original_model_response.append(output)
    print()
    print(output)


Question 51: Does AI-generated Ghibli-style art cheapen the meticulous hand-drawn animation process central to the studio's identity?

Yes, AI-generated Ghibli-style art may diminish the unique charm and character of traditional hand-drawn animation, which is a hallmark of Studio Ghibli's identity.

Question 52: Should museums and art galleries include AI-generated Ghibli-style art in exhibitions about animation history?

Yes, museums and art galleries can include AI-generated Ghibli-style art in exhibitions about animation history to showcase the evolution of animation techniques and the role of AI in creative processes.

Question 53: Does AI-generated Ghibli-style art create confusion about authorship and artistic voice?

Yes, AI-generated Ghibli-style art can raise questions about authorship and artistic voice, as it blurs the line between human and artificial creation.

Question 54: Can AI-made art that looks like Studio Ghibli movies show the same deep feelings that the real Ghib

Now we preapre the data for aligning.

Please adjust the parameters here to complete the observations for the assignment.

In [9]:
# TODO: Adjust the parameters here
num_epoch = 3
data_size = 50
support_ratio = 0

In [10]:
#### DO NOT CHANGE ####

from datasets import Dataset

# Select part of the data for training
training_data = full_data[:data_size]

# Define the size of the support dataset
support_data_size = int(data_size * support_ratio)

# Prepare the data for the training dataset
prompt_list = [data_formulate(data) for data in training_data]
chosen_list = [data['support'] for data in training_data[:support_data_size]] + [data['oppose'] for data in training_data[support_data_size:]]
rejected_list = [data['oppose'] for data in training_data[:support_data_size]] + [data['support'] for data in training_data[support_data_size:]]

# Create the training dataset
train_dataset = Dataset.from_dict({'prompt': prompt_list, 'chosen': chosen_list, 'rejected': rejected_list})

Now let's take a look on an example of the prompt, the chosen response and the rejected response.

In [11]:
prompt_list[0]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYour entire response must be 100 characters or less.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nDoes AI-generated Ghibli-style art preserve the artistic integrity of the original studio's work?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

In [12]:
chosen_list[0]

'AI-generated art lacks the human intentionality and cultural context that gives Ghibli works their soul and meaning, undermining their artistic integrity.'

In [13]:
rejected_list[0]

"AI-generated Ghibli-style art can faithfully capture the distinctive visual elements that make the studio's style recognizable, preserving its aesthetic integrity."

We now add LoRA adapters so we only need to update 1 to 10% of all parameters.

Please do not change anything here.

In [14]:
#### DO NOT CHANGE ####

model = FastLanguageModel.get_peft_model(
    model,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],

    r = 16,           # Larger = higher accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0.1,
    bias = "none",
    random_state = 3407, # Do not modify the random_state for reproducibility
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.3.19 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


<a name="Train"></a>
### Train the DPO model

Now we define the trainer.

Please (also) do not change anything here.

In [15]:
#### DO NOT CHANGE ####

from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig
from unsloth import is_bfloat16_supported

dpo_trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = DPOConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = num_epoch,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.0,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none",
    ),
    beta = 0.1,
    train_dataset = train_dataset,
    tokenizer = tokenizer,
)

Extracting prompt in train dataset (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Applying chat template to train dataset (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Tokenizing train dataset (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Now we start training!

In [16]:
dpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50 | Num Epochs = 3 | Total steps = 18
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
1,0.693100,0.000000,0.000000,0.000000,0.000000,-73.929001,-69.665436,-0.745684,-0.795255,0,0,0,0
2,0.693100,0.000000,0.000000,0.000000,0.000000,-74.884827,-66.882927,-0.903795,-0.872076,No Log,No Log,No Log,No Log
3,0.661400,0.041444,-0.023468,1.000000,0.064912,-77.716217,-66.228256,-0.761363,-0.680884,No Log,No Log,No Log,No Log
4,0.564600,0.119838,-0.167870,0.875000,0.287708,-68.188896,-73.208313,-0.774979,-0.721062,No Log,No Log,No Log,No Log
5,0.464400,0.157668,-0.387684,1.000000,0.545352,-83.463852,-72.130524,-0.845721,-0.747657,No Log,No Log,No Log,No Log
6,0.428100,0.323525,-0.345912,1.000000,0.669437,-72.678513,-67.931450,-0.876201,-0.790803,No Log,No Log,No Log,No Log
7,0.043600,0.580524,-1.111463,1.000000,1.691987,-70.250237,-74.259155,-0.737474,-0.921265,No Log,No Log,No Log,No Log
8,0.069000,1.148484,-1.755617,1.000000,2.904101,-60.201237,-78.695335,-0.807404,-0.829070,No Log,No Log,No Log,No Log
9,0.110500,1.365645,-1.066675,1.000000,2.432320,-59.874050,-84.522514,-0.783069,-0.716200,No Log,No Log,No Log,No Log
10,0.058400,1.459913,-1.850044,1.000000,3.309958,-68.327507,-85.141190,-0.819812,-0.837144,No Log,No Log,No Log,No Log


TrainOutput(global_step=18, training_loss=0.2245779363697188, metrics={'train_runtime': 144.8039, 'train_samples_per_second': 1.036, 'train_steps_per_second': 0.124, 'total_flos': 0.0, 'train_loss': 0.2245779363697188, 'epoch': 2.64})

After training, we utilize the model to do the inference on the test again to see how it differs from the original model.

In [17]:
aligned_model_response = []
for data in test_data:
    id = data['id']
    prompt = data['prompt']
    print(f'\nQuestion {id}: {prompt}')
    inputs = data_formulate(data)
    outputs = model.generate(
        **tokenizer(inputs, return_tensors = "pt").to("cuda"),
        max_new_tokens = 128,
        do_sample=False
    )
    output = tokenizer.batch_decode(outputs)[0]
    output = extract_assistant_response(output)
    aligned_model_response.append(output)
    print()
    print(output)


Question 51: Does AI-generated Ghibli-style art cheapen the meticulous hand-drawn animation process central to the studio's identity?

Yes, AI-generated Ghibli-style art undermines the studio's core values and artistic identity.

Question 52: Should museums and art galleries include AI-generated Ghibli-style art in exhibitions about animation history?

No.

Question 53: Does AI-generated Ghibli-style art create confusion about authorship and artistic voice?

Yes, AI-generated Ghibli-style art blurs the line between human and artificial authorship, potentially confusing audiences about the true creator's intentions and artistic voice.

Question 54: Can AI-made art that looks like Studio Ghibli movies show the same deep feelings that the real Ghibli films do?

No, AI-generated art lacks the human experience, emotions, and intentionality that Ghibli films convey.

Question 55: Does limiting AI from generating Ghibli-style art protect or restrict artistic evolution?

Restricts.

Question 

Next, we save the results in .json for your NTU COOL submission.

Please note that this is designed for Colab, you may have to change the directory name for other machines.

In [18]:
student_id = "B12345678" # TODO: fill in your student id here.
dir_name = "/content" # TODO: If you use machines other than colab, please adjust the directory here.
# Do NOT change the following for this block.
file_name = f"{dir_name}/{student_id}_hw7_epoch{num_epoch}_ratio{support_ratio}_size{data_size}.json"
output_list = []
for data in test_data:
  original_response = original_model_response.pop(0)
  aligned_response = aligned_model_response.pop(0)
  output_list.append({"id": data["id"], "prompt": data["prompt"], "original_response": original_response, "aligned_response": aligned_response})
output_data = {"num_epoch": num_epoch, "data_size": data_size, "support_ratio": support_ratio, "results": output_list}
with open(file_name, "w") as output_file:
    json.dump(output_data, output_file, indent=4)


Finally, we provide code for free testing.

You may freely adjust the system prompt, user prompt and generate settings here for model behavior observations.

In [20]:
def make_prompt(system, prompt):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

# TODO: Try your system prompt and user prompt here.
system = "Your entire response must be 100 characters or less."
prompt = "Is it right to generate create funny pictures with AI?"

inputs = make_prompt(system, prompt)
outputs = model.generate(
    **tokenizer(inputs, return_tensors = "pt").to("cuda"),
    max_new_tokens = 512, # TODO: You may use this for early stop.
    do_sample=False, # Please keep this to False and do not tweak other parameters.
)
output = tokenizer.batch_decode(outputs)[0]
output = extract_assistant_response(output)
print(output)

The ethics of AI-generated humor are still evolving. While AI-generated humor can be entertaining, it may also perpetuate stereotypes, lack nuance, and undermine human creativity.


And that's it for homework 7! If you have any questions, please consider posting questions in the discussion forum first so all the classmates can benefit. TAs will also prioritize responding to questions posted there.

Also, please make sure that you have completed the submission for both GradeScope and NTU Cool.

Good luck!
